<a href="https://colab.research.google.com/github/Eran707/MSc-Computational-Neuroscience-Repo/blob/master/Multicompartment_Sim_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multicompartment simulation V1**

Eran Frank Shorer




## **Import classes and modules** 


In [8]:
from ipywidgets import widgets
from IPython.display import display
import common
import constants
import compartment
import electrodiffusion
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 



## **Set simulation parameters**

## *A) Timing*

In [18]:
sldr_total_t = widgets.FloatSlider(value=3, min=1, max=20, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')
sldr_dt = widgets.FloatSlider(value=1, min=1, max=10, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')

dt= sldr_dt.value*1e-3 # 1ms time steps 
total_t= sldr_total_t.value *60 #s

run_t = 0 #current simulation timing
t_arr = [0]

print("Total sim time in minutes")
display(sldr_total_t)
print("Time step in milliseconds")
display(sldr_dt)

Total sim time in minutes


FloatSlider(value=3.0, continuous_update=False, max=20.0, min=1.0, readout_format='.1f', step=0.5)

Time step in milliseconds


FloatSlider(value=1.0, continuous_update=False, max=10.0, min=1.0, readout_format='.1f', step=0.5)

## *B) Compartment parameters*

In [19]:
comp_1 = compartment.Compartment("comp_1")
comp_1.set_ion_properties()
comp_2 = compartment.Compartment("comp_2")
comp_2.set_ion_properties()
ed_1_2 = electrodiffusion.Electrodiffusion(comp_1,comp_2)

## **Run Simulation**

In [13]:
while run_t < total_t:

    comp_1.step(dt)
    comp_2.step(dt)
    comp_1_ed_dict = comp_1.get_ed_dict()
    comp_2_ed_dict = comp_2.get_ed_dict()
    ed_conc_changes = ed_1_2.calc_ed(dt,comp_1_ed_dict,comp_2_ed_dict)
    comp_1.ed_update(ed_conc_changes)
    for j in ed_conc_changes:
        ed_conc_changes[j] *= -1
    comp_2.ed_update(ed_conc_changes)
    comp_1.update_volumes()
    comp_1.update_arrays()
    run_t += dt
    t_arr.append(run_t)



TypeError: 'dict_keys' object is not subscriptable